# M. Soban Anjum - BSDSF21A007

---

## Implement Nearest Neighbor

In [1]:
import numpy as np
from PIL import Image

image = Image.open('hehe.jpg')
input_image = np.array(image)

def nearest_neighbor_interpolation(input_image, new_width, new_height):
    orig_height, orig_width = input_image.shape[:2]

    resized_image = np.zeros((new_height, new_width, 3), dtype=np.uint8)

    # Compute scaling factors
    x_scale = new_width / orig_width
    y_scale = new_height / orig_height

    for y in range(new_height):
        for x in range(new_width):
            # Find the nearest neighbor pixel in the original image
            orig_x = int(x / x_scale)
            orig_y = int(y / y_scale)

            # Assign the pixel value
            resized_image[y, x] = input_image[orig_y, orig_x]

    return resized_image

resized_img = nearest_neighbor_interpolation(input_image, 800, 600)
resized_image = Image.fromarray(resized_img)
resized_image.show()  
resized_image.save("resized_image_Nearest_Neighbour.jpg")


---

## Bilinear Interpolation and Resizing

In [2]:
import numpy as np
from PIL import Image

image = Image.open('hehe.jpg')
input_image = np.array(image)

def bilinear_interpolation(input_image, new_width, new_height):
    orig_height, orig_width = input_image.shape[:2]
    
    resized_image = np.zeros((new_height, new_width, 3), dtype=np.uint8)
    
    x_scale = new_width / orig_width
    y_scale = new_height / orig_height

    for y in range(new_height):
        for x in range(new_width):
            orig_x = x / x_scale
            orig_y = y / y_scale

            x1 = int(orig_x)
            y1 = int(orig_y)
            x2 = min(x1 + 1, orig_width - 1)
            y2 = min(y1 + 1, orig_height - 1)

            x_weight = orig_x - x1
            y_weight = orig_y - y1

            # Interpolate along x for both top and bottom pairs
            top = (1 - x_weight) * input_image[y1, x1] + x_weight * input_image[y1, x2]
            bottom = (1 - x_weight) * input_image[y2, x1] + x_weight * input_image[y2, x2]

            # Interpolate along y
            interpolated_pixel = (1 - y_weight) * top + y_weight * bottom
            resized_image[y, x] = interpolated_pixel

    return resized_image

new_width = 800
new_height = 600

resized_img_bilinear = bilinear_interpolation(input_image, new_width, new_height)
resized_image_bilinear = Image.fromarray(resized_img_bilinear)
resized_image_bilinear.show()  # Display resized image
resized_image_bilinear.save('resized_image_bilinear.jpg')  # Save resized image


---

## Implement Convolution

In [3]:
def convolve_image(input_image, kernel):
    image_height, image_width = input_image.shape[:2]
    
    kernel_height, kernel_width = kernel.shape
    
    # Padding the image to handle edges
    pad_height = kernel_height // 2
    pad_width = kernel_width // 2
    padded_image = np.pad(input_image, ((pad_height, pad_height), (pad_width, pad_width), (0, 0)), mode='constant')
    
    # Create empty array for the convolved image
    convolved_image = np.zeros_like(input_image)
    
    # Loop over every pixel in the image
    for y in range(image_height):
        for x in range(image_width):
            # Extract the region of interest
            region = padded_image[y:y+kernel_height, x:x+kernel_width]
            
            # Perform element-wise multiplication and sum the results
            for channel in range(3):  # Loop over RGB channels
                convolved_image[y, x, channel] = np.sum(region[:, :, channel] * kernel)
    
    convolved_image = np.clip(convolved_image, 0, 255)
    
    return convolved_image


sobel_kernel = np.array([[-1, 0, 1],
                         [-2, 0, 2],
                         [-1, 0, 1]])

convolved_img = convolve_image(input_image, sobel_kernel)
convolved_image = Image.fromarray(convolved_img.astype(np.uint8))
convolved_image.show() 
convolved_image.save('convolved_image.jpg')  


--- 

## Gaussian Filter --  ( input image- Apply Gaussian filter-Output filtered image)

In [4]:
def gaussian_kernel(size, sigma=1):
    """Generates a Gaussian kernel."""
    kernel = np.fromfunction(lambda x, y: (1/(2*np.pi*sigma**2)) * 
                             np.exp(-((x-(size-1)/2)**2 + (y-(size-1)/2)**2) / (2*sigma**2)),
                             (size, size))
    return kernel / np.sum(kernel)

def apply_gaussian_filter(input_image, kernel_size=5, sigma=1):
    kernel = gaussian_kernel(kernel_size, sigma)
    return convolve_image(input_image, kernel)

gaussian_filtered_image = apply_gaussian_filter(input_image, kernel_size=5, sigma=1)
gaussian_filtered_image = Image.fromarray(gaussian_filtered_image.astype(np.uint8))
gaussian_filtered_image.show()  
gaussian_filtered_image.save('gaussian_filtered_image.jpg')  


---